In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi, cos, sin, sqrt
%matplotlib notebook

fig = plt.figure()

plt.xlabel('x')
plt.ylabel('y')
plt.title('test')
plt.margins(x=0.1,y=0.1,tight=False)

speed = 100.0
ax_max = 500.0/1.5
ay_max = 500.0/1.5
part_max = 0.5
delta_max = 0.2

def gen_test_path():
    x = 5.0
    y = -3.0
    path = []
    last_a = 0
    if 1:
        for l, delta_a in [(10, 30), (10, 20), (10, 150), (10, -90), (10, 20), (10, 45), (10, 160)]:
            last_a += delta_a
            a1 = last_a / 180.0 * pi
            dx = l * cos(a1)
            dy = l * sin(a1)
            dt = l / speed
            path.append([x, y, x + dx, y + dy, dt, 'i'])
            x += dx
            y += dy
    else:
        for i in range(40):
            l, delta_a = 5,9
            last_a += delta_a
            a1 = last_a / 180.0 * pi
            dx = l * cos(a1)
            dy = l * sin(a1)
            dt = l / speed
            path.append([x, y, x + dx, y + dy, dt, 'i'])
            x += dx
            y += dy


    path.append([x, y, x, y, 1, 'l'])

    path[0][5] = 'f'
    
    return path

path = gen_test_path()

fig = plt.figure()

x = [a for a,b,_,_,_,_ in path]
y = [b for a,b,_,_,_,_ in path]
plt.plot(x,y, "g.-")

In [ ]:
def cubic_coeff_trapezoid(x0, v0, xn, vn, deltaT):
    d = x0
    c = v0
    b = 3 * (xn - x0) / (deltaT * deltaT) - (2 * v0 + vn) / deltaT
    a = (vn - v0) / (3.0 * (deltaT * deltaT)) - (2.0 * b) / (3.0 * deltaT)
    return np.array([a, b, c, d])

import scipy.optimize as optimize

def quad_coeff(x0, v0, xn, vn, deltaT):
    start = (0,0,0,v0,x0)
    a,b,c,d,e = start

    def f(a_b):
        #print("try", coeff)

        a, b = list(a_b)
        coeff = (a, b, c, d, e)
        start_x = quad_x(coeff, 0)
        stop_x = quad_x(coeff, deltaT)
        start_v = quad_v(coeff, 0)
        stop_v = quad_v(coeff, deltaT)
        start_a = quad_a(coeff, 0)
        stop_a = quad_a(coeff, deltaT)

        delta = 0
        #delta += abs(x0 - start_x)
        delta += 100*(xn - stop_x)**2
        #delta += abs((v0 - start_v))
        delta += 10*(vn - stop_v)**2
        #delta += abs(start_a)
        delta += stop_a**2
        #print("delta", delta)
        return delta
    
    res = optimize.minimize(f, (a, b), tol=1e-13, method='Powell', jac=False)
    print("res", res)
    coeff = (res.x[0], res.x[1], c, d, e)
    print("start:", start)
    print("coeff:", coeff)

    start_x = quad_x(coeff, 0)
    stop_x = quad_x(coeff, deltaT)
    start_v = quad_v(coeff, 0)
    stop_v = quad_v(coeff, deltaT)
    start_a = quad_a(coeff, 0)
    stop_a = quad_a(coeff, deltaT)
    print("x0", start_x - x0)
    print("xn", stop_x - xn)
    print("v0", start_v - v0)
    print("vn", stop_v - vn)
    print("a0", start_a)
    print("an", stop_a)

    return coeff

def cubic_x(coefs, t):
    a, b, c, d = list(coefs)
    return a * (t * t * t) + b * (t * t) + c * t + d


def cubic_v(coefs, t):
    a, b, c, d = list(coefs)
    return 3.0 * a * (t * t) + 2.0 * b * t + c


def cubic_a(coefs, t):
    a, b, c, d = list(coefs)
    return 6.0 * a * t + 2.0 * b

##########
def quad_x(coefs, t):
    a, b, c, d, e = list(coefs)
    return a * (t * t * t * t) + b * (t * t * t) + c * t * t + d * t + e


def quad_v(coefs, t):
    a, b, c, d, e = list(coefs)
    return 4 * a * (t * t * t) + 3 * b * (t * t) + 2 * c * t + d


def quad_a(coefs, t):
    a, b, c, d, e = list(coefs)
    return 12 * a * (t * t) + 6 * b * t + 2 * c

def quad_j(coefs, t):
    a, b, c, d, e = list(coefs)
    return 24 * a * t + 6 * b




In [ ]:
def do_split(path):
    x, y = path[0][:2]
    vx = 0.0
    vy = 0.0
    need_slow = 1
    k_slow = 1.0

    while need_slow:
        need_slow = 0
        points = []
        last_dt = 0
        for (x1, y1, x2, y2, dt, fli) in path:
            dt = dt * k_slow
            nvx = (x2 - x1) / dt
            nvy = (y2 - y1) / dt
            dvx = nvx - vx
            dvy = nvy - vy
            dta = max(abs(dvx / ax_max), abs(dvy / ay_max))

            if fli == "i":
                dta = dta / 2
                ka = dta / dt
            else:
                ka = dta / dt / 2

            #print fli, dvx, dvy, dt, dta, dta/dt
            last_ka = dta / (last_dt or 1)

            if fli == "f":
                points.append((x1, y1, 0, 0, 0))
                points.append((x1 + (x2 - x1) * ka, y1 + (y2 - y1) * ka, nvx, nvy, dta))
                dta = dta / 2
            elif fli == "i":
                ins_ka = 1.0 - last_ka - ka
                div_ka = 1

                if ins_ka > 1.99:
                    div_ka = 4
                    for i in range(1,4):
                        points.append((last_x2 - (last_x2 - last_x1) * (last_ka + ins_ka* i/4),
                            last_y2 - (last_y2 - last_y1) * (last_ka + ins_ka*i/4),
                            vx, vy, (last_dt - last_dta - dta)/4))

                points.append((last_x2 - (last_x2 - last_x1) * last_ka,
                               last_y2 - (last_y2 - last_y1) * last_ka,
                               vx, vy, (last_dt - last_dta - dta)/div_ka))
                points.append((x1 + (x2 - x1) * ka,
                               y1 + (y2 - y1) * ka,
                               nvx, nvy, dta * 2))
            elif fli == "l":
                points.append((last_x2 - (last_x2 - last_x1) * last_ka / 2,
                               last_y2 - (last_y2 - last_y1) * last_ka / 2,
                               vx, vy, last_dt - last_dta - dta / 2))
                points.append((x2, y2,
                               0, 0, dta))
            if fli == "i":
                if ka > part_max:
                    need_slow = 1
                if last_dt and last_ka > part_max:
                    need_slow = 1

                x0, y0, vx0, vy0, _ = points[-2]
                xn, yn, vxn, vyn, sdt = points[-1]
                coefs_x = cubic_coeff_trapezoid(x0, vx0, xn, vxn, sdt)
                coefs_y = cubic_coeff_trapezoid(y0, vy0, yn, vyn, sdt)

                xm = cubic_x(coefs_x, sdt / 2)
                ym = cubic_x(coefs_y, sdt / 2)
                delta = sqrt((xm - x1) * (xm - x1) + (ym - y1) * (ym - y1))
                print delta

                if delta > delta_max:
                    need_slow = 1
            else:
                if ka > 0.5:
                    need_slow = 1
                if last_dt and last_ka > 0.5:
                    need_slow = 1

            last_x1 = x1
            last_y1 = y1
            last_x2 = x2
            last_y2 = y2
            last_dt = dt
            last_dta = dta
            vx = nvx
            vy = nvy

        #need_slow = 0
        if need_slow:
            k_slow = k_slow * 1.1
            print "need slowdown, new k =", k_slow, "speed = ", speed / k_slow

    return points

points = do_split(path)

In [ ]:
fig = plt.figure()

xp = [a for a,b,_,_,_,_ in path]
yp = [b for a,b,_,_,_,_ in path]

x = [a for a,b,_,_,_ in points]
y = [b for a,b,_,_,_ in points]

plt.plot(xp, yp, 'r-')
plt.plot(x, y, "g.-")


In [ ]:
from collections import namedtuple
emu_point = namedtuple("emu_point", "t x y vx vy ax ay")

def emu(points):
    emu = []
    last_p = points[0]
    global_t = 0.0
    for p in points[1:]:
        x0, y0, vx0, vy0, _ = last_p
        x1, y1, vx1, vy1, dt = p

        coefs_x = quad_coeff(x0, vx0, x1, vx1, dt)
        coefs_y = quad_coeff(y0, vy0, y1, vy1, dt)

        step = dt / 10
        
        for i in range(11):
            t = i * step
            if i == 10:
                t = t - 0.00001
            
            x = quad_x(coefs_x, t)
            y = quad_x(coefs_y, t)
            vx = quad_v(coefs_x, t)
            vy = quad_v(coefs_y, t)
            ax = quad_a(coefs_x, t)
            ay = quad_a(coefs_y, t)

            emu.append(emu_point(global_t + t, x, y, vx, vy, ax, ay))

        #last_p = p
        x = quad_x(coefs_x, dt)
        y = quad_x(coefs_y, dt)
        vx = quad_v(coefs_x, dt)
        vy = quad_v(coefs_y, dt)
        last_p = x, y, vx, vy, dt
        global_t += dt
        
    return emu

emu_data = emu(points)

emu_data


In [ ]:
fig = plt.figure()

xp = [a for a,b,_,_,_ in points]
yp = [b for a,b,_,_,_ in points]

x = [a.x for a in emu_data]
y = [a.y for a in emu_data]

plt.plot(xp, yp, 'r.-')
plt.plot(x, y, "g.-")

In [ ]:
fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.vx for a in emu_data]
y = [a.vy for a in emu_data]


plt.plot(t, x, 'r.-')
plt.plot(t, y, "g.-")

fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.ax for a in emu_data]
y = [a.ay for a in emu_data]


plt.plot(t, x, 'r.-')
plt.plot(t, y, "g.-")



In [ ]:
ax_auto = 1500
ay_auto = 1500
a_delta = 0.5

def quad_coeff_auto(xv0, yv0, xv1, yv1):
    k = 0.2
    ak = 1.5

    dtx = abs(xv1 - xv0)/ax_auto*ak
    dty = abs(yv1 - yv0)/ay_auto*ak
    dt = max(dtx, dty)
    if dt == 0:
        dt = 1

    start = (dt, k)
    
    def make_coeff(pos):
        dt, k = list(pos)
        k = abs(k)
        dt = abs(dt)
        
        adj_xv0 = xv0 * k
        adj_yv0 = yv0 * k
        adj_xv1 = xv1 * k
        adj_yv1 = yv1 * k

        x0 = - adj_xv0 * dt/2
        y0 = - adj_yv0 * dt/2

        ax = (xv1 - xv0)/dt*ak
        ay = (yv1 - yv0)/dt*ak
    
        xdj = - ax/3/dt/dt
        xj = -2 * xdj * dt

        ydj = - ay/3/dt/dt
        yj = -2 * ydj * dt

        e = x0
        d = adj_xv0
        c = 0
        b = xj
        a = xdj
        
        xcoeff = (a, b, c, d, e)

        e = y0
        d = adj_yv0
        c = 0
        b = yj
        a = ydj
        
        ycoeff = (a, b, c, d, e)

        return xcoeff, ycoeff, dt, k

    def f(pos):
        xcoeff, ycoeff, dt, k = make_coeff(pos)

        adj_xv0 = xv0 * k
        adj_yv0 = yv0 * k
        adj_xv1 = xv1 * k
        adj_yv1 = yv1 * k
        
        stop_x = quad_x(xcoeff, dt)
        middle_x = quad_x(xcoeff, dt/2)
        stop_vx = quad_v(xcoeff, dt)
        stop_ax = quad_a(xcoeff, dt)
        middle_ax = quad_a(xcoeff, dt/2)

        stop_y = quad_x(ycoeff, dt)
        middle_y = quad_x(ycoeff, dt/2)
        stop_vy = quad_v(ycoeff, dt)
        stop_ay = quad_a(ycoeff, dt)
        middle_ay = quad_a(ycoeff, dt/2)

        delta = 0

        error2 = middle_x**2 + middle_y**2
        delta += 1000*abs((error2-a_delta**2))

        if abs(middle_ax) > ax_auto:
            delta += 1000*(abs(middle_ax) - ax_auto)**2

        if abs(middle_ay) > ay_auto:
            delta += 1000*(abs(middle_ay) - ay_auto)**2
        
        if abs(middle_ax)>abs(middle_ay):
            if abs(middle_ax) < ax_auto:
                delta += 100*(abs(middle_ax) - ax_auto)**2
        else:
            if abs(middle_ay) < ay_auto:
                delta += 100*(abs(middle_ay) - ay_auto)**2
            
        #print("delta", delta)
        return delta
    
    res = optimize.minimize(f, start, tol=1e-13, method='Powell', jac=False,
                            #bounds = [(None, None), (None, None), (0.0, None), (0.0, None)],
                            options = {"maxiter": 1000000, "maxfev": 10000000 })
    print("res", res)
    print("start:", start)
    print("finish:", res.x)

    return make_coeff(res.x)



In [ ]:
x1, y1, x2, y2, dt1, _ = path[1]
_, _, x3, y3, dt2, _ = path[2]



lp = path[0][:]
lp[2] = lp[0]
lp[3] = lp[1]

emu_data = []

global_t = 0.0
ks = 1

for cp in path:
    x1, y1, x2, y2, dt1, _ = lp
    _, _, x3, y3, dt2, _ = cp
    
    print("lp", lp)
    print("cp", cp)

    lp = cp
    
    xv0 = (x2 - x1)/dt1*ks
    yv0 = (y2 - y1)/dt1*ks
    xv1 = (x3 - x2)/dt2*ks
    yv1 = (y3 - y2)/dt2*ks

    print('xv', xv0, xv1)
    print('yv', yv0, yv1)

    coefs_x, coefs_y, dt, k = quad_coeff_auto(xv0, yv0, xv1, yv1)
    coefs_x = list(coefs_x)
    coefs_y = list(coefs_y)

    coefs_x[4] += x2
    coefs_y[4] += y2

    tx0 = quad_x(coefs_x, 0)
    ty0 = quad_x(coefs_y, 0)
    tx1 = quad_x(coefs_x, dt)
    ty1 = quad_x(coefs_y, dt)

    subs = 100

    step = dt / subs

    for i in range(subs+1):
        t = i * step

        x = quad_x(coefs_x, t)
        y = quad_x(coefs_y, t)
        vx = quad_v(coefs_x, t)
        vy = quad_v(coefs_y, t)
        ax = quad_a(coefs_x, t)
        ay = quad_a(coefs_y, t)

        emu_data.append(emu_point(t+global_t, x, y, vx, vy, ax, ay))

    global_t += dt



In [ ]:

fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.x for a in emu_data]
y = [a.y for a in emu_data]

plt.plot(x, y, 'r.-')

xp = [a for a,b,_,_,_,_ in path]
yp = [b for a,b,_,_,_,_ in path]

plt.plot(xp, yp, 'g.-')


fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.vx for a in emu_data]
y = [a.vy for a in emu_data]


plt.plot(t, x, 'r.-')
plt.plot(t, y, "g.-")

fig = plt.figure()

t = [a.t for a in emu_data]
x = [a.ax for a in emu_data]
y = [a.ay for a in emu_data]


plt.plot(t, x, 'r.-')
plt.plot(t, y, "g.-")